# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("output_data/weather_data.csv")
weather_data

,City,Country,Latitude,Longitude,Date,Cloudiness (%),Humidity (%),Max Temperature (°C),Wind Speed (m/s)
0,Bengkulu,ID,-3.8004,102.2655,1619514452,38,76,28.86,2.69
1,Kavieng,PG,-2.5744,150.7967,1619514472,98,77,27.98,3.69
2,Loandjili,CG,-4.7561,11.8578,1619514409,40,79,29.00,3.60
3,Qaanaaq,GL,77.4840,-69.3632,1619514472,99,71,-0.12,3.79
4,Barra de Tecoanapa,MX,16.5167,-98.7500,1619514472,73,72,25.68,2.72
...,...,...,...,...,...,...,...,...,...
500,Astoria,US,46.1879,-123.8313,1619514418,90,93,8.89,1.54
501,Pacific Grove,US,36.6177,-121.9166,1619514678,40,71,11.00,2.06
502,Weiser,US,44.2510,-116.9693,1619514678,1,86,2.78,1.54
503,Bereket,TM,39.2446,55.5154,1619514678,30,14,35.92,3.97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Store Humidity as weight, find maxiumum Humidity
humidity = weather_data["Humidity (%)"]
humidity_max = humidity.max()

# Store Latitude and Longitude as locations
locations = weather_data[["Latitude", "Longitude"]]

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create Heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = humidity_max, point_radius = 3)

# Add Heat layer and display
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Find cities with max temperature between 21°C and 27°C, wind speed less than 5 m/s and 0 cloudiness
narrowed_weather_data = weather_data.loc[(weather_data["Max Temperature (°C)"] > 21) & (weather_data["Max Temperature (°C)"] < 27) & (weather_data["Cloudiness (%)"] == 0) & (weather_data["Wind Speed (m/s)"] < 5), :]

# Drop any rows with null values
narrowed_weather_data = narrowed_weather_data.dropna(how='any')
narrowed_weather_data.reset_index(inplace=True)
del narrowed_weather_data['index']

# Display narrowed down cities
narrowed_weather_data

,City,Country,Latitude,Longitude,Date,Cloudiness (%),Humidity (%),Max Temperature (°C),Wind Speed (m/s)
0,Vila Velha,BR,-20.3297,-40.2925,1619514495,0,94,23.00,1.03
1,Maceió,BR,-9.6658,-35.7353,1619514417,0,100,22.00,1.54
2,Alice Springs,AU,-23.7000,133.8833,1619514597,0,20,23.33,4.12
3,San Javier,ES,37.8063,-0.8374,1619514497,0,72,22.22,2.06


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into hotel_df variable
hotel_df = narrowed_weather_data

# Add Hotel Name column
hotel_df["Hotel Name"] = " "

# Display new DataFrame
hotel_df

,City,Country,Latitude,Longitude,Date,Cloudiness (%),Humidity (%),Max Temperature (°C),Wind Speed (m/s),Hotel Name
0,Vila Velha,BR,-20.3297,-40.2925,1619514495,0,94,23.00,1.03,
1,Maceió,BR,-9.6658,-35.7353,1619514417,0,100,22.00,1.54,
2,Alice Springs,AU,-23.7000,133.8833,1619514597,0,20,23.33,4.12,
3,San Javier,ES,37.8063,-0.8374,1619514497,0,72,22.22,2.06,


In [7]:
# Empty list to hold hotel names
hotel_ls = []

# Construct URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters
params = {"type": "hotel",
          "keyword": "hotel",
          "radius": 5000,
          "key" : g_key}

# Iterate through cities in hotel_df
for index, row in hotel_df.iterrows():
    
    # Find latitude and longitude for each city in DataFrame
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # Add location to parameters
    params["location"] = f"{lat},{lng}"
    
    # Construct URL and make API request
    response = requests.get(base_url, params = params).json()
    
    try:
        hotel_ls.append(response["results"][0]["name"])
    
    except:
        hotel_ls.append("NaN")

In [8]:
hotel_df["Hotel Name"] = hotel_ls
hotel_df

,City,Country,Latitude,Longitude,Date,Cloudiness (%),Humidity (%),Max Temperature (°C),Wind Speed (m/s),Hotel Name
0,Vila Velha,BR,-20.3297,-40.2925,1619514495,0,94,23.00,1.03,Hotel Senac Ilha do Boi
1,Maceió,BR,-9.6658,-35.7353,1619514417,0,100,22.00,1.54,Meridiano Hotel
2,Alice Springs,AU,-23.7000,133.8833,1619514597,0,20,23.33,4.12,DoubleTree by Hilton Hotel Alice Springs
3,San Javier,ES,37.8063,-0.8374,1619514497,0,72,22.22,2.06,Thalasia Costa de Murcia


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))